# Einfache Transformationen

In [ ]:
# Bibliotheken einmalig installieren

%pip install pyspark pandas

In [ ]:
!wget -O https://raw.githubusercontent.com/Fuenfgeld/2022TeamADataEngineeringBC/pull-everything-together/Pr%C3%A4sentationen/02-Einfache%20Transformationen/create_data.py

In [ ]:
import sqlite3
import pandas as pd
import pyspark.sql.functions as pysparkFunctions 
import os

os.system("python3 create_data.py")

In [ ]:
## Datenbankdaten in Dataframe schreiben
connection_obj = sqlite3.connect('Diddly_Squat_Farm.db')
cursor_obj = connection_obj.cursor()

# Tabelle Fields ausgeben
for row in cursor_obj.execute('SELECT * FROM fields'):
        print(row)

# Tabelle Fields in Dataframe df_fields schreiben
df_fields = pd.read_sql_query("SELECT * FROM fields", connection_obj)
print('\n\nDies ist das erzeugte Dataframe:\n\n', df_fields)

connection_obj.close()

In [ ]:
## (Py)Spark starten

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
## JSON-/CSV-Dateien in Dataframes schreiben
df_fru = spark.read.options(header='True', multiline='True', inferSchema='True').csv('Fruits.csv')
df_veg = spark.read.option("multiline",True).json('Vegetables.json')

# erstellte Dataframes ausgeben (10 Einträge ausgeben)
df_fru.show(10)
df_veg.show(10)


In [ ]:
## Zeilen abfragen: Jeremy überlegt welche Pflanze er anbaut und schaut sich dafür die Zucchini genauer an

df_veg_lokal = df_veg.collect()  # eigentlich unnötig, da bei uns ales lokal läuft
print(f"Zucchini in der ersten Woche: {df_veg_lokal[8]}\n")

In [ ]:
## Spalten abfragen:	Jeremy will sich Übersicht über generellen Wasserverbrauch der Gemüsesorten verschaffen -> dazu begutachtet er die Spalte 'Wasserverbrauch'  
df_veg.select("crop", "water_consumption").show()

In [ ]:
## DataFrames Zusammenfügen: Jeremy überlegt ob er Gemüse oder Obst anbauen soll, 
## deshalb verschafft er sich einen Gesamtübersicht/Vergleich aller Obst-&Pflanzensorten: Obst- & Gemüsesorten konkatenieren

## Um die beiden Dataframes kombinieren zu können müssen sie in das gleiche Format gebracht werden.
## Achtung hier verändert sich der semantische Sinn der Spalte. Dies ist hier egal, weil wir sie weiter unten löschen.
df_veg_correct_revenue = df_veg.withColumn("revenue", pysparkFunctions.column("revenue.net"))
df_veg_correct_revenue.show(5)

df_fru_veg = df_fru.union(df_veg_correct_revenue)
df_fru_veg.show()

In [ ]:
## Dupliakte entfernen: Wie Jeremy gerade aufgefallen ist, enthällt die Tabelle noch Duplikate.
## (Siehe z.B. Woche 2 der Erdbeeren) Diese sollen nun entfernt werden.

df_fru_veg = df_fru_veg.dropDuplicates()
df_fru_veg.show()

In [ ]:
## Spalten entfernen:   Um einen klareren Blick auf die Daten zu erhalten, entfernt Jeremy alle Spalten
## die nicht über den Wasserverbrauch aussagen

df_fru_veg = df_fru_veg.drop(*('field', 'revenue', 'week', 'yield_per_sqm'))
df_fru_veg.show()

In [ ]:
## Aufräumen:	Für Jeremy sind jetzt natürlich nur Einträge relevant, die einen Wasserverbrauch beinhalten.

df_fru_veg = df_fru_veg.dropna()
df_fru_veg.show()

In [ ]:
## Wasserverbrauch anzeigen:    Jeremy möchte wissen, wie der gesamte Wasserverbrauch im letzten Jahr war.

df_fru_veg.agg(pysparkFunctions.sum("water_consumption")).collect()

In [ ]:
## Wasserverbrauch pro Pfanzenart anzeigen: Jeremy möchte noch wissen, wie der Wasserverbrauch für die unterschiedlichen Gewächsarten ist.

df_fru_veg.filter(df_fru_veg.crop == "strawberries").agg(pysparkFunctions.sum("water_consumption")).collect()

In [ ]:
## Daten speichern: Um langfristig auf die Daten zurückgreifen zu können, speichern wird jetzt noch die Rohdaten in einer Datenbank.

connection = sqlite3.connect('result.sqlite')

df_raw_data = df_fru.union(df_veg).drop_duplicates()
df_raw_data.toPandas().to_sql('water_consumption', connection, if_exists='replace', index=True)

print(connection.execute('SELECT * FROM water_consumption;').fetchall())

connection.close()


**Übungen**

1. Dieses Notebook enthält noch eine weitere JSON Datei namens `Corn.json` mit Daten zu Gedreide. Werte den Wasserverbrauch nur für das Gedreide aus.
2. Ermittle wie viel Wasser pro Feld eingesetzt wurde.
3. Ermittle den gesamten Wasserverbrauch für alle Pfanzen.